In [32]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [33]:
import progressbar
import hdfs
from os import path
client2 = hdfs.InsecureClient('http://localhost:9870', user='root')
client2.makedirs("/random-dot")
client2.makedirs("/random-dot/train-data")
hdfs_path = '/random-dot/train-data'
train_data_folder = f'C:\\Users\\rmsry\\Data_Analytics\\CA3\\random-dot\\magiceye_gen\\shapes\\stereograms'

In [ ]:
total_images = 20000
bar = progressbar.ProgressBar(max_value=total_images)
barprogress = 0

def progress_bar(file, b):
    global barprogress
    global bar
    if(b == -1):
        barprogress += 1
        bar.update(barprogress)

def upload_shape(shape_name:str, hdfs_path='/random-dot/train-data'):
    hdfs_path = f'/random-dot/train-data'
    local_path = f'{train_data_folder}\\{shape_name}'
    # Create shape folder
    client2.makedirs(hdfs_path)
    # Upload stereograms images
    try:
        client2.upload(hdfs_path=hdfs_path, local_path=local_path, progress=progress_bar)
    except hdfs.util.HdfsError:
        global barprogress
        global bar
        barprogress += 1
        bar.update(barprogress)
        pass
    
    
def upload_all():
    upload_shape("circle")
    upload_shape("square")
    upload_shape("star")
    upload_shape("triangle")
    

# Upload all images to hadoop hdfs

run upload_all() to move all images into hdfs

In [ ]:
#upload_all()

# Upload training images to hdfs
traing-shapes.tgz 2.9Gb

In [ ]:
# Compressed file
compress_train_images = f'{train_data_folder}\\train-shapes.tgz'
if path.exists(compress_train_images) is False:
    make_tarfile(compress_train_images, train_data_folder)

In [ ]:
# Upload file to hadoop hdfs
client2.upload(hdfs_path=hdfs_path, local_path=compress_train_images)

# Upload subset of images for development 

In [18]:
import progressbar
from os import path, mkdir
import shutil
import tensorflow as tf

train_data_folder = f'C:\\Users\\rmsry\\Data_Analytics\\CA3\\random-dot\\magiceye_gen\\shapes\\stereograms'
batch_size = 32
img_height = 1024
img_width = 1024

In [7]:
#take only 10% of images for development
dataset_images = tf.keras.utils.image_dataset_from_directory(
  train_data_folder,
  validation_split=0.9,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 18435 files belonging to 4 classes.
Using 1844 files for training.


In [8]:
dataset_images.class_names

['circle', 'square', 'star', 'triangle']

In [22]:
#set progress bar
bar = progressbar.ProgressBar(max_value=len(dataset_images.file_paths))
barprogress = 0
targetDir = f'C:\\Users\\rmsry\\Data_Analytics\\CA3\\random-dot\\magiceye_gen\\shapes\\stereograms-dev'

# create folder structure
if path.exists(targetDir) is False:
    mkdir(targetDir)
    for c in dataset_images.class_names:
        mkdir(f'{targetDir}\\{c}')

        # copy file for development
for file in dataset_images.file_paths:
    destFile = file.replace(train_data_folder,targetDir)
    shutil.copy(file,destFile)
    barprogress += 1
    bar.update(barprogress)


 97% (1798 of 1844) |################### | Elapsed Time: 0:00:01 ETA:   0:00:00

In [29]:
# Upload compressed file
compress_development_images = f'{targetDir}\\train-shapes-dev.tgz'
if path.exists(compress_development_images) is False:
    make_tarfile(compress_development_images, targetDir)

In [38]:
# Upload file to hadoop hdfs
client2.upload(hdfs_path=hdfs_path, local_path=compress_development_images)

'/random-dot/train-data/train-shapes-dev.tgz'